In [2]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt

# Memuat CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Mengubah kolom string ke float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Find the  min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

# Mengubah skala kolom kumpulan data ke kisaran 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
    # X_norm = (X - X_min) / (X_max - X_min)
    #skala diubah dengan rumus
    # data - nilai minimum / nilai maksimum - nilai minimum

# Membagi datase kedalam k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Menghitung akurasi dalam persen
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluasi algoritma menggunakan cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            #row_copy[-1] = None                          #edited - nilai prediksi dikosongkan pada data testing
            row_copy[0] = None                            #kolom ke-0 (pertama) diberi nilai None
        predicted = algorithm(train_set, test_set, *args)
        #actual = [row[-1] for row in fold]               #edited - nilai class asli diambil
        actual = [row[0] for row in fold]                 #kolom ke-0 (pertama) berisi nilai class asli
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    #print("trainset: ", train_set)
    #print("testset: ", test_set)
    return scores

# Hitung jarak antara data training dengan data testing
def euclidean_distance(row1, row2):
    distance = 0.0
    # for i in range(len(row1)-1):                        #edited - perulangan dimulai dari i = 1, bukan 0. Karena data pertama adalah nilai class
    #     distance += (row1[i] - row2[i])**2              #edited
    for i in range(1, len(row1)-1):
         distance += (row1[i] - row2[i])**2
    print("distance ", sqrt(distance))
    return sqrt(distance)

# Dapatkan data netihbors (yang paling mirip)
def get_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    #print("neighbors: ", neighbors)
    return neighbors

# Membuat prediksi dengan neighbors
def predict_classification(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    #output_values = [row[-1] for row in neighbors]       #edited - output berada pada data pertama (row[0]), bukan pada data terakhir row([1]) seperti sebelumnya
    output_values = [row[0] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

# kNN algorithm
def k_nearest_neighbors(train, test, num_neighbors):
    predictions = list()
    for row in test:
        output = predict_classification(train, row, num_neighbors)
        predictions.append(output)
    return(predictions)

# mencoba kNN ke dalam dataset
seed(1)
filename = 'datasetlks.csv'
dataset = load_csv(filename)
#for i in range(len(dataset[0])-1):                     #edited perulangan dimulai dari i = 1 atau kolom ke-2
for i in range(1, len(dataset[0])-1):
    str_column_to_float(dataset, i)
# Mengubah kelas kolom ke integer
# str_column_to_int(dataset, len(dataset[0])-1)         #edited nilai yg diubah ke integer adalah nilai pada kolom ke-0, bukan kolom terakhir
str_column_to_int(dataset, 0)
# Evaluasi algoritma
n_folds = 10
num_neighbors = 5
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)

Streaming output truncated to the last 5000 lines.
distance  13.738537040019947
distance  30.42381797210863
distance  23.753395546742368
distance  13.080986201353474
distance  20.855970368218305
distance  3.5599016840356703
distance  18.483752324676935
distance  13.007324859478217
distance  6.818988194739745
distance  7.953313774773381
distance  12.731790918798502
distance  7.339243830259354
distance  62.10420597672914
distance  6.323693540961643
distance  16.379010959151348
distance  7.554991727328363
distance  7.319856555971572
distance  4.283094675582132
distance  3.785234470941001
distance  5.30164125530953
distance  13.6106649359978
distance  22.592233178683333
distance  4.049654306234052
distance  5.301518650349163
distance  11.240053380656162
distance  3.0016828613296234
distance  7.640130888931157
distance  9.024134307511165
distance  7.998018504604749
distance  9.364074967662315
distance  13.838489801997905
distance  14.463125526662623
distance  3.565992148056413
distance  12.

In [3]:
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Mean Accuracy: 83.529%
